In [4]:
from NeuroPy import NeuroPy
import time
import threading

# Replace with the correct COM port or device path
# Example: COM3 for Windows, /dev/rfcomm0 for Linux
PORT = "COM3"

# Initialize headset
headset = NeuroPy(PORT, 57600)

# Callback functions
def attention_callback(attention_value):
    print(f"[ATTENTION] {attention_value}")

def meditation_callback(meditation_value):
    print(f"[MEDITATION] {meditation_value}")

def raw_data_callback(raw_value):
    print(f"[RAW EEG] {raw_value}")

# Assign callbacks
headset.setCallBack("attention", attention_callback)
headset.setCallBack("meditation", meditation_callback)
headset.setCallBack("rawValue", raw_data_callback)

# Run headset in a background thread
def start_headset():
    headset.start()

# Start in a separate thread to avoid blocking
t = threading.Thread(target=start_headset)
t.daemon = True
t.start()

print("Reading data from MindWave Mobile 2. Press Ctrl+C to stop.")
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\nStopping data stream...")
    headset.stop()


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (NeuroPy.py, line 119)